In [5]:
from datasets import load_dataset
import numpy as np
import os
import argparse
from peft import PeftModel
from utils import *
from huggingface_params import cache_dir, use_auth_token
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm 
import time 

In [6]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', 
                                          cache_dir=cache_dir,
                                          use_auth_token = use_auth_token,
                                          trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained('gpt2', 
                                            cache_dir=cache_dir,
                                            use_auth_token = use_auth_token,
                                            trust_remote_code=True)

model.generation_config.pad_token_id = tokenizer.pad_token_id
model.to("cuda")
model.eval()

dataset = load_dataset("gsm8k", "main")

test_questions = dataset["test"]["question"]
test_answers = dataset["test"]['answer']
eval_questions = test_questions
eval_questions = [question + "\nAnswer:" for question in eval_questions]
eval_answers = test_answers

n = 5
temperature=0.8
max_tokens=512
top_p=0.95
seed=0
stop="\nDone."
bad_words_ids = [tokenizer(stop).input_ids]

/home/kbaek/miniconda3/envs/sphere/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/kbaek/miniconda3/envs/sphere/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [3]:
start_time = time.time()
samples = []
for x in test_questions:
  tokens = tokenizer(x, return_tensors='pt').to('cuda')
  prediction = tokenizer.decode(model.generate(**tokens, temperature=temperature, max_new_tokens=max_tokens, top_p=top_p, bad_words_ids=bad_words_ids)[0])
  samples.append(prediction)
  break

end_time = time.time()

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
